In [1]:
import fitz # PyMuPDF
from collections import defaultdict

# 폰트 프로파일링 코드
def profile_fonts(pdf_path):
    doc = fitz.open(pdf_path)
    fonts = defaultdict(list)
    
    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" not in block: continue
            for line in block["lines"]:
                for span in line["spans"]:
                    font_name = span["font"]
                    text_sample = span["text"].strip()
                    if text_sample and font_name not in fonts:
                        fonts[font_name] = text_sample
                        
    print("--- PDF 폰트 프로파일링 결과 ---")
    for font, sample in fonts.items():
        print(f"폰트 이름: {font}, 샘플 텍스트: '{sample}'")

# 프로파일링 실행
pdf_file = "2020년 국제해사기구 전문용어집_페이지 편집본.pdf"
profile_fonts(pdf_file)

--- PDF 폰트 프로파일링 결과 ---
폰트 이름: AuctionGothicBold, 샘플 텍스트: 'IMO 회의 및 조직 용어'
폰트 이름: AuctionGothicLight, 샘플 텍스트: 'Credential'
폰트 이름: KoPubDotumBold, 샘플 텍스트: '신임장'
폰트 이름: KoPubDotumLight, 샘플 텍스트: '신임장이란 특정인을 외교 사절로 파견하는 취지와 그 사람의 신분을'
폰트 이름: AuctionGothicMedium, 샘플 텍스트: '(IMO)'
폰트 이름: KoPubDotumMedium, 샘플 텍스트: '구 분'
폰트 이름: HCRBatang, 샘플 텍스트: '▶'
폰트 이름: HaansoftBatang, 샘플 텍스트: '․'
폰트 이름: X26073B90, 샘플 텍스트: '<출처: https://m.blog.naver.com/koreamof/221678579938>'
폰트 이름: Ymjo420, 샘플 텍스트: '*'


In [42]:
import fitz
import pandas as pd
import re

# --- 글꼴 힌트 ---
FONT_ENGLISH = "AuctionGothicLight"
FONT_KOREAN = "KoPubDotumBold"
FONT_DESC   = "KoPubDotumLight"

def clean_text(text):
    text = re.sub(r'[▶∙❶❷❸❹❺❻❼￭*]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_glossary(pdf_path):
    doc = fitz.open(pdf_path)
    all_entries = []

    for page in doc:
        # 모든 span 수집
        spans = [s for b in page.get_text("dict")["blocks"]
                   for l in b.get("lines", [])
                   for s in l.get("spans", [])]

        # 페이지 내 앵커(영단어) 순회
        i = 0
        while i < len(spans):
            span = spans[i]
            text = clean_text(span["text"])
            if not text:
                i += 1
                continue

            if FONT_ENGLISH in span["font"]:  # 영단어 발견
                eng_word = text

                # 오른쪽/같은 줄에서 한국어 단어 찾기
                kor_word = ""
                for s in spans[i+1:i+5]:  # 근처 5 span만 탐색
                    if FONT_KOREAN in s["font"]:
                        kor_word = clean_text(s["text"])
                        break

                # 아래쪽 설명 찾기 (다음 영어 단어 전까지)
                desc_parts = []
                j = i + 1
                while j < len(spans):
                    s = spans[j]
                    if FONT_ENGLISH in s["font"]:
                        break  # 다음 영단어 나오면 설명 종료
                    if FONT_DESC in s["font"]:
                        desc_parts.append(clean_text(s["text"]))
                    j += 1
                description = " ".join(desc_parts)

                if eng_word and kor_word and description:
                    all_entries.append({
                        "영단어": eng_word,
                        "한국단어": kor_word,
                        "설명": description
                    })

                i = j  # 다음 탐색 위치 점프
            else:
                i += 1

    doc.close()
    return pd.DataFrame(all_entries)

# --- 실행 ---
pdf_file = "2020년 국제해사기구 전문용어집_페이지 편집본.pdf"
df = extract_glossary(pdf_file)

# 저장
csv_file = "try3.csv"
df.to_csv(csv_file, sep='|', index=False, encoding='utf-8-sig')

print(f"✅ 추출 완료! {len(df)}개의 항목을 '{csv_file}'에 저장했습니다.")
print(df.head(10))


✅ 추출 완료! 624개의 항목을 'try3.csv'에 저장했습니다.
                  영단어      한국단어  \
0          Credential       신임장   
1         Secretariat       사무국   
2            Assembly        총회   
3             Council       이사회   
4   Secretary-General      사무총장   
5      Council Reform    이사회 개편   
6  Chair / Vice-Chair  의장 / 부의장   
7                 SMC   고위관리위원회   
8  World Maritime Day  세계 해사의 날   
9      Maritime Prize   국제 해사 상   

                                                  설명  
0  신임장이란 특정인을 외교 사절로 파견하는 취지와 그 사람의 신분을 상대 국가에 통고...  
1  사무국은 조직·단체에서 인사, 총무 등의 운영사무를 담당하는 부서를 지칭함. 국제해...  
2  국제기구에서 총회는 모든 회원국이 모여서 의제를 토의하고 어떠한 사안에 관한 최종 ...  
3  국제해사기구(IMO)의 실질적인 집행기관으로서 역할을 하고 있음. 총회가 최종 의사...  
4  사무국의 업무를 총괄하는 직위로서 국제해사기구(IMO) 수장을 의미함. 역대 IMO...  
5  기존 40개국으로 구성된 IMO 이사국 제도의 개편을 의미하며, 구체적으로 이사국 ...  
6  회의 진행을 총괄하는 직위를 지칭하며 초기에는 의장을 Chairman 명칭을 사용하...  
7  SMC (Senior Management Committee), 사무총장과 사무국의 ...  
8  국제해사기구(IMO)는 매년 9월 마지막 목요일을 ‘World Maritime Da...  
9  국제해사기구(IMO) 목적달성에 가장 이